In [3]:
from numpy.linalg import inv
import numpy as np
from numpy.linalg import norm
def simplex_step(A, b, c, iB, iN, xB,  irule=0, Binv=""):
    if not isinstance(iB, np.ndarray):
        iB = np.array(iB)
    if not isinstance(iN, np.ndarray):
        iN = np.array(iN)
    if c.shape[1] > 1:
        c = c.T # make c a column vector
    # iB begin with index 1
    iB = iB - 1
    if isinstance(Binv, str):
        Binv = inv(A[:,iB])
        w_t = np.matmul(c[iB,0].T, Binv)
    else:
        w_t = np.matmul(c[iB,0].T, Binv)
    reduced_cost = c.T - np.matmul(w_t, A)
    if np.sum(reduced_cost>=0) < reduced_cost.shape[1]:
        print("not optimal yet")
        if irule == 0:
            j = np.argmin(reduced_cost)
        u = np.matmul(Binv, A[:,j])
        if np.sum(u > 0) == 0:
            print("optimal cost is negative infinity")
            istatus = 16
            return [istatus]
        else:
            # minimal ratio test
            u = np.asarray(u).reshape(-1,)
            idx = np.asarray(np.argwhere(u>0)).reshape(-1)
            delta = np.asarray(xB).reshape(-1)[idx] / u[idx]
            l = np.argmin(delta)
            delta_ =delta[l]
            for i in range(A.shape[0]):
                if i != l:
                    #xB[i,0] += xB[l,0] * (-A[i,j]/A[l,j])
                    xB[iB[i],0] -= delta_*u[i]
            xB[j,0] = delta_
            print("an iteration finished")
            istatus = 0
            iB[l] = j
            Binv = inv(A[:,iB])
            iB = iB + 1 # recover the index of basic variables
            iN = np.setdiff1d(range(1, A.shape[1]+1), iB)
            return [istatus, iB, iN, xB, Binv]
    else:
        print("reach optimal")
        istatus = -1
        return [istatus, iB, iN, xB, Binv]

In [7]:
from numpy.linalg import inv
import numpy as np


def simplex_step(A, b, c, iB, iN, xB, irule=0, Binv=""):
    """
    Take a single simplex method step for the linear program

      min:    c*x
      ST:     Ax=b
              x>=0,

    where A is an (m,n) matrix.

    That is, given a basic feasible vector vector described by the variables
    iB,iN,XB,

    Input Parameters:

        A - (n,m) constraint matrix
        b - (m,1) POSITIVE vector appearing in the constraint equation above
        c - (1,n) vector giving the coefficients of the objective function

        iB - (1,m) integer vector specifying the indices of the basic
            variables at the beginning of the simplex step
        iN - (1,n-m) integer vector specying the indices of the nonbasic
            variables at the beginning of the simplex step
        xB - (m,1) vector specifying the values of the basic
            variables at the beginning of the simplex step

        irule - integer parameter speciying which pivot rule to use:
            irule = 0 indicates that the smallest coefficient rule should be
                used
            irule = 1 indicates that Bland's rule should be used

    Output Parameters:

        istatus - integer parameter reporting the condition of the
            istatus = 0  indicates normal simplex method step
            completed
            istatus = 16 indicates the program is unbounded
            istatus = -1 indicates an optimal feasible vector has been
            found

        iB - integer vector specifying the m indices of the basic variables
            after the simplex step
        iN - integer vector specifying the n-m indices of the nonbasic
            variables after the simplex step
        xB - vector of length m specifying the values of the basic
            variables after the simplex step
    """
    if not isinstance(iB, np.ndarray):
        iB = np.array(iB)
    if not isinstance(iN, np.ndarray):
        iN = np.array(iN)
    if c.shape[1] > 1:
        c = c.T  # make c a column vector
    iB = iB - 1  # iB begin with index 1
    if isinstance(Binv, str):
        Binv = inv(A[:, iB])
        w_t = np.matmul(c[iB, 0].T, Binv)
    else:
        w_t = np.matmul(c[iB, 0].T, Binv)
    reduced_cost = c.T - np.matmul(w_t, A)
    if np.sum(reduced_cost >= 0) < reduced_cost.shape[1]:
        #print("not optimal yet")
        if irule == 0:
            j = np.argmin(reduced_cost)
        elif irule == 1:
            for idx, flag in enumerate((x < 0)):
                if flag:
                    j = idx
                    break
        else:
            raise ValueError("incorrect irule!")
        u = np.matmul(Binv, A[:, j])
        if np.sum(u > 0) == 0:
            print("optimal cost is negative infinity")
            istatus = 16
            return [istatus]
        else:
            # minimal ratio test
            u = np.asarray(u).reshape(-1,)
            idx = np.asarray(np.argwhere(u > 0)).reshape(-1)
            delta = np.asarray(xB).reshape(-1)[idx] / u[idx]
            l = idx[np.argmin(delta)]#np.argmin(delta)
            delta_ = min(delta)
            for i in range(A.shape[0]):
                if i != l:
                    xB[i,0] -= delta_ * (u[i] / u[l]) * xB[l,0]
                else:
                    xB[i] = delta_
            print("an iteration finished")
            istatus = 0
            iB[l] = j
            Binv = inv(A[:, iB])
            print(xB)
            iB = iB + 1  # recover the index of basic variables
            iN = np.setdiff1d(range(1, A.shape[1]+1), iB)
            return [istatus, iB, iN, xB, Binv]
    else:
        print("reach optimal")
        istatus = -1
        return [istatus, iB, iN, xB, Binv]

In [8]:
A1 = np.matrix([[1, 1, 2],
                [1, 1, 1],
                [1, 1, 1]],dtype = np.float64)

A = np.hstack((np.eye(3), A1))

b = np.matrix([[1],
            [2],
            [3]],dtype = np.float64)


iB = [1,2,3]
iN = [4,5,6]
xB = np.matrix(np.copy(b))
c  = np.matrix([[0,0,0,-1,-2,1]],dtype = np.float64)

# form an invertible matrix B and modify the problem
B=np.matrix([[4, 1, 0],
            [1, -2, -1],
            [1, 2, 4]],dtype = np.float64)
A=B*A
b=B*b
# modify c

N=A[:,[index_N-1 for index_N in iN]]
c1=np.matrix([1, 1, 1],dtype = np.float64)
c2=c1*B.I*N+c[:,[index_N-1 for index_N in iN]]


# take a step with the first rule
irule = 0
[istatus,iB,iN,x,Binv] = simplex_step(A,b,c,iB,iN,xB,irule)


X = np.zeros((6,1),dtype = np.float64)
X[[(b-1) for b in iB]] = xB

if (istatus != 0):
    print('INCORRECT ISTATUS!\n')

if (norm(X-np.matrix([[0],[1],[2],[0],[1],[0]]) > 1e-10)):
    print('INCORRECT STEP!\n')


if (norm(np.array(sorted(iN))-np.array([1, 4, 6])) > 1e-10):
    print('iN incorrect!\n')


if (norm(np.array(sorted(iB))-np.array([2, 3, 5])) > 1e-10):
    print('iB incorrect!\n')

an iteration finished
[[1.]
 [1.]
 [2.]]


In [9]:
iB,iN,xB

(array([5, 2, 3]), array([1, 4, 6]), matrix([[1.],
         [1.],
         [2.]]))

In [54]:
u = np.array([-1,2,3,-2])
xB = np.array([3,4,3,2])

In [55]:
idx = np.asarray(np.argwhere(u > 0)).reshape(-1)
delta = np.asarray(xB).reshape(-1)[idx] / u[idx]
idx, delta

(array([1, 2]), array([2., 1.]))

In [58]:
l = idx[np.argmin(delta)]
l

2

In [60]:
delta_ = min(delta)
delta_

1.0

In [61]:
for i in range(4):
    if i != l: 
        xB[i] -= delta_ * u[i]
xB[l] = delta_

In [62]:
xB

array([4, 2, 1, 4])

In [32]:
reduced_cost = np.array([[ 0, 0, 0, -1, -2,  1]])

for idx, flag in enumerate((reduced_cost[0] < 0)):
    if flag:
        j = idx
        break

In [31]:
reduced_cost.reshape(reduced_cost.shape[0],-1)

array([[ 0,  0,  0, -1, -2,  1]])

In [56]:
import numpy as np
from simplex_step import * 
from simplex_init import simplex_init
def find_negative_index(b):
    row_idxs = []
    for row_idx, element in enumerate(b):
        if element < 0:
            row_idxs.append(row_idx)
    return row_idxs
A = np.matrix([[1,2,3,0],
               [-1,2,6,0],
               [0,4,9,0],
              [0,0,3,1]],dtype = np.float64)
b = np.matrix([[3],
               [2],
               [5],[1]],dtype = np.float64)

c = np.matrix([[1,1,1,0]],dtype = np.float64)

In [66]:
A_new, b_new = A, b
A_new[find_negative_index(b)] = -A[find_negative_index(b)]
b_new[find_negative_index(b)] = -b[find_negative_index(b)]
A_new = np.hstack((A_new, np.eye(b.shape[0])))
# problem setup
c_phase_I = np.zeros(A_new.shape[1]).reshape(1, -1)
c_phase_I[0, c.shape[1]:] = np.ones(b.shape[0])
iB = np.arange(c.shape[1], c.shape[1] + b.shape[0]) + 1  # index begin with 1 for input
iN = np.arange(0, c.shape[1]) + 1
xB = np.matrix(np.copy(b))
istatus_step = 1000
while istatus_step != -1:
    try:
        istatus_step, iB, iN, xB, Binv = simplex_step(A_new, b_new, c_phase_I, iB, iN, xB, irule=0)
    except np.linalg.LinAlgError:
        raise ValueError("iB cannot form a basis!")
    if istatus_step == 16:
        istatus, iB, iN, xB = 4, None, None, None
iB = iB - 1
optimal_cost = np.matmul(c_phase_I[0, iB].reshape(1, -2), xB)

In [67]:
optimal_cost

matrix([[0.]])

In [68]:
if optimal_cost == 0:
    #print("optimal basis is found!")
    istatus = 0
    artificial_idx = np.arange(c.shape[1], c.shape[1] + b.shape[0])
    if len(np.intersect1d(artificial_idx, iB)) == 0:
        print("no artificial variable in the final basis")
    else:
        artificial_in_basis = np.intersect1d(artificial_idx, iB)
        tableau = np.matmul(Binv, A_new)
        for xl in artificial_in_basis:
            row_l = tableau[np.where(iB == xl), :c.shape[1]]
            if np.sum(row_l) == 0:
                xB = np.delete(xB, np.where(iB == xl))
                iB = np.delete(iB, np.where(iB == xl))
        iN = np.setdiff1d(range(c.shape[1]), iB)
        iB = iB + 1
        iN = iN + 1

In [3]:
from simplex_method import *
from simplex_init import *
import numpy as np
from numpy.linalg import inv
A = np.matrix([[1,1,1,0],
               [-1,1,2,0],
               [0,2,3,0],[0,0,1,1]],dtype = np.float64)
b = np.matrix([[6],
               [4],
               [10],[2]],dtype = np.float64)

c = np.matrix([[-1,2,-3,0]],dtype = np.float64)

In [4]:
init_status, iB, iN, xB = simplex_init(A, b, c)

In [5]:
init_status, iB, iN, xB

(0, array([2, 3, 1]), array([4]), matrix([[2., 2., 2.]]))

array([array([2, 3, 1]), array([4])], dtype=object)

In [11]:
# simplex_method(A, b, c, irule=1)
# simplex_step(A, b, c, iB, iN, xB, irule=0, Binv="")
A[:, np.vstack(iB,iN)]

TypeError: vstack() takes 1 positional argument but 2 were given

In [6]:
istatus_step, iB, iN, xB, _ = simplex_step(A, b, c, iB, iN, xB, irule=0, Binv="")

IndexError: index 6 is out of bounds for axis 1 with size 4

In [11]:
from simplex_method import *
from simplex_init import *
import numpy as np
from numpy.linalg import inv
A = np.matrix([[1,2,3,0],
               [-1,2,6,0],
               [0,4,9,0],[0,0,3,1]],dtype = np.float64)
b = np.matrix([[3],
               [2],
               [5],[1]],dtype = np.float64)

c = np.matrix([[1,1,1,0]],dtype = np.float64)

In [12]:
init_status, iB, iN, xB = simplex_init(A, b, c)

In [13]:
iB, iN, xB

(array([2, 3, 1]), array([4]), matrix([[0.5       , 0.33333333, 1.        ]]))

In [15]:
iN

array([4])

In [3]:
np.eye(b.shape[0])

array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]])

In [4]:
A

matrix([[ 1.,  1.,  1.,  0.],
        [-1.,  1.,  2.,  0.],
        [ 0.,  2.,  3.,  0.],
        [ 0.,  0.,  1.,  1.]])

In [26]:
A = np.matrix([[1,1,1,0],
               [-1,1,2,0],
               [0,2,3,0],[0,0,1,1]])

In [14]:
from simplex_method import *
A = np.matrix([[1,1,1,0],
               [-1,1,2,0],
               [0,2,3,0],[0,0,1,1]],dtype = np.float64)
b = np.matrix([[6],
               [4],
               [10],[2]],dtype = np.float64)

c = np.matrix([[-1,2,-3,0]],dtype = np.float64)

In [15]:
simplex_method(A,b,c, irule=0)

LinAlgError: Last 2 dimensions of the array must be square

In [16]:
from simplex_method import *
import numpy as np
from numpy.linalg import inv
A = np.matrix([[1,1,-1,0,0],
               [-1,1,0,-1,0],
               [0,1,0,0,1]],dtype = np.float64)
b = np.matrix([[2],
               [1],
               [3]],dtype = np.float64)

c = np.matrix([[1,-2,0,0,0]],dtype = np.float64)

In [17]:
init_status, iB, iN, xB, tableau = simplex_init_modified(A, b, c)
simplex_method(A,b,c, irule=0)

(0, array([[0.],
        [3.],
        [1.],
        [2.],
        [0.]]), matrix([[-6.]]), array([4, 2, 3]), array([1, 5]), matrix([[2.],
         [3.],
         [1.]]))

In [18]:
import numpy as np 
from simplex_method_new import *
from numpy.linalg import inv
A = np.matrix([[1,2,3,0],
               [-1,2,6,0],
               [0,4,9,0],
              [0,0,3,1]],dtype = np.float64)
b = np.matrix([[3],
               [2],
               [5],[1]],dtype = np.float64)

c = np.matrix([[1,1,1,0]],dtype = np.float64)

In [19]:
import numpy as np
import random
from simplex_method import simplex_method
R = np.matrix([[4, 1,  1],
               [-1, 2,  1],
               [1, 1, -1]], dtype=np.float64)

# form a vector b which is in the span of R
b = R*np.matrix([[1],
                 [2],
                 [1]], dtype=np.float64)

B = np.matrix([[1, 1, 1],
               [1, 1, 0],
               [1, 0, 0]], dtype=np.float64)
A = np.hstack((R, B))

#form a random permutation
p = list(range(0, 6))
random.shuffle(p)
A = A[:, p]

c = np.matrix([[-2, 1, 1, -1, -1, -1]], dtype=np.float64)
c = c[:, p]

# test
irule = 0
[istatus, X, eta, iB, iN, xB] = simplex_method(A, b, c, irule)